In [22]:
import pandas as pd
import numpy as np

In [23]:
data = pd.read_excel('InsumosProyecto_Omega.xlsx')
tabla_mortalidad = pd.read_excel('tabla_mortalidad.xlsx')['qx']

In [24]:
def edad_asegurable(edad,sexo, status):
    if sexo == "F" and status == 'NF':
        edad -= 5
    elif sexo=="F":
        edad -= 3
    elif status == 'NF':
        edad -= 2
    return edad

In [25]:
def seguro_temporal(edad, sexo, status, tasa, SA_T, cobertura):
    edad = edad_asegurable(edad, sexo, status)
    k = np.array([x for x in range(cobertura)])
    x_k = edad + k
    q_x = np.array(tabla_mortalidad.loc[x_k[0]:x_k[-1]])
    v_k1 = np.array([(1+tasa)**(-1-x) for x in range(cobertura)])
    kp_x = [1]
    for i in range(cobertura-1):
        valor = kp_x[i] * (1-q_x[i])
        kp_x.append(valor)
    PNU = sum(q_x * v_k1 * kp_x) * SA_T
    return np.round(PNU,2)

In [26]:
def seguro_vitalicio(edad, sexo, status, tasa, SA_T):
    omega = 110
    edad = edad_asegurable(edad, sexo, status)
    cobertura = omega-edad+1
    k = np.array([x for x in range(cobertura)])
    x_k = edad + k
    q_x = np.array(tabla_mortalidad.loc[x_k[0]:x_k[-1]])
    v_k1 = np.array([(1+tasa)**(-1-x) for x in range(cobertura)])
    kp_x = [1]
    for i in range(cobertura-1):
        valor = kp_x[i] * (1-q_x[i])
        kp_x.append(valor)
    PNU = sum(q_x * v_k1 * kp_x) * SA_T
    return np.round(PNU,2)

In [27]:
def seguro_dotal(edad, sexo, status, tasa, SA, cobertura):
    edad = edad_asegurable(edad, sexo, status)
    k = np.array([x for x in range(cobertura)])
    x_k = edad + k
    q_x = np.array(tabla_mortalidad.loc[x_k[0]:x_k[-1]])
    npx = np.product([1-q_x[i] for i in range(len(q_x))])
    v_k = (1+tasa)**(-cobertura)
    PNU = SA*v_k*npx
    return np.round(PNU,2)

In [28]:
def seguro_dotal_mixto(edad, sexo, status, tasa, SA_T, SA_D, cobertura):
    s_temporal = seguro_temporal(edad, sexo, status, tasa, SA_T, cobertura)
    s_dotal = seguro_dotal(edad, sexo, status, tasa, SA_D, cobertura)
    PNU = s_temporal + s_dotal
    return np.round(PNU,2)

In [29]:
def calcular_PNU(tipo_seguro,edad, sexo, status, tasa, SA_T, SA_D, cobertura):
    if tipo_seguro == 'T':
        PNU = seguro_temporal(edad, sexo, status, tasa, SA_T, cobertura)
    elif tipo_seguro == 'V':
        PNU = seguro_vitalicio(edad, sexo, status, tasa, SA_T)
    elif tipo_seguro == 'DM':
        PNU = seguro_dotal_mixto(edad, sexo, status, tasa, SA_T, SA_D, cobertura)
    return np.round(PNU,2)

In [30]:
def anualidad(edad, sexo, status, tasa, pagos):
    edad = edad_asegurable(edad, sexo, status)
    k = np.array([x for x in range(pagos)])
    x_k = edad + k
    q_x = np.array(tabla_mortalidad.loc[x_k[0]:x_k[-1]])
    v_k1 = np.array([(1+tasa)**(-x) for x in range(pagos)])
    kp_x = [1]
    for i in range(pagos-1):
            valor = kp_x[i] * (1-q_x[i])
            kp_x.append(valor)
    anualidad = sum( v_k1 * kp_x) 
    return anualidad


In [31]:
data['PNU'] =  data.apply(lambda x: calcular_PNU(x.PROD, x.ECON, x.SEXO, x.STATUS, x.T_P, x.SAF, x.SAD, x.COB), axis = 1)
data['Anualidad'] = data.apply(lambda x: anualidad(x.ECON, x.SEXO, x.STATUS, x.T_P, x.PAG), axis = 1)

In [32]:
GA = 0.12
CA = 0.12
U = 0.07

In [33]:
data['PT'] = data['PNU'] / (1-GA-CA-U)
data['PT'] = data['PT'].map(lambda x: np.round(x,2))

In [34]:
data['PA'] = data['PT']/ data['Anualidad']
data['PA'] = data['PA'].map(lambda x: np.round(x,2))

In [35]:
data['Anualidad'] = data['Anualidad'].map(lambda x: np.round(x,3))

In [42]:
data.to_csv('PRIMAS.csv', index=False)